# 2. Analisi Esplorativa dei Dati (EDA)

**Obiettivo:** Analizzare i dati puliti per scoprire pattern, identificare trend e anomalie, e comprendere le relazioni tra le diverse statistiche dei giocatori.

**Fasi:**
1.  **Caricamento Dati:** Lettura del DataFrame pulito (formato Parquet).
2.  **Analisi Leader Statistici:** Identificazione dei top performer per punti, rimbalzi e assist in una stagione recente (2023).
3.  **Analisi di Distribuzione:** Visualizzazione della distribuzione dei punti per partita (PPG) per capire la concentrazione del talento offensivo.
4.  **Analisi di Correlazione:** Creazione di una heatmap per esplorare le relazioni lineari tra le principali statistiche.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import desc

# Aggiunge la root del progetto al sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importazione delle utility, della configurazione e inizializzazione di Spark
from src.utils.helpers import get_spark_session
from src.config.spark_config import SPARK_CONFIG

# Inizializza la sessione Spark
spark = get_spark_session(
    app_name="NBA_EDA",
    driver_memory=SPARK_CONFIG["driver_memory"]
)

# Impostazioni standard per i grafici
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

### Fase 1: Caricamento dei Dati Puliti

In [ ]:
# Percorso del file Parquet contenente i dati puliti
cleaned_df_path = "../data/processed/players_cleaned.parquet"
df = spark.read.parquet(cleaned_df_path)

# Stampa dello schema per verificare il corretto caricamento
print("Schema del DataFrame caricato:")
df.printSchema()

### Fase 2: Analisi dei Leader Statistici (Stagione 2023)

Filtriamo i dati per la stagione 2023 per analizzare le performance dei giocatori attuali e identificare i leader nelle categorie principali.

In [ ]:
# Filtra il DataFrame per la stagione 2023
df_2023 = df.filter(df.season == 2023)

# Identifica e visualizza i top 10 marcatori
print("Top 10 Marcatori (Punti Totali) - Stagione 2023")
df_2023.orderBy(desc("pts")).select("player", "pts").show(10)

# Identifica e visualizza i top 10 rimbalzisti
print("Top 10 Rimbalzisti (Rimbalzi Totali) - Stagione 2023")
df_2023.orderBy(desc("trb")).select("player", "trb").show(10)

# Identifica e visualizza i top 10 assistmen
print("Top 10 Assistmen (Assist Totali) - Stagione 2023")
df_2023.orderBy(desc("ast")).select("player", "ast").show(10)

### Fase 3: Distribuzione dei Punti per Partita (Stagione 2023)

Creiamo un istogramma per visualizzare come si distribuisce la media punti per partita (PPG) tra tutti i giocatori. Questo ci aiuta a capire quanti giocatori sono di élite, quanti sono nella media e quanti sono giocatori di ruolo.

In [ ]:
# Calcola i punti per partita (PPG) e converte in Pandas per la visualizzazione
ppg_df = df_2023.withColumn("ppg", df_2023.pts / df_2023.g).select("ppg").toPandas()

# Creazione dell'istogramma della distribuzione
plt.figure(figsize=(14, 7))
sns.histplot(ppg_df['ppg'], bins=30, kde=True)
plt.title('Distribuzione dei Punti per Partita (PPG) - Stagione 2023', fontsize=16)
plt.xlabel('Punti per Partita (PPG)', fontsize=12)
plt.ylabel('Numero di Giocatori', fontsize=12)
plt.show()

### Fase 4: Correlazione tra le Statistiche di Base

Utilizziamo una heatmap per visualizzare la matrice di correlazione. Questo ci permette di identificare rapidamente quali statistiche tendono a muoversi insieme (es. punti e assist).

In [ ]:
# Seleziona le colonne numeriche di interesse e converte in Pandas
stats_df = df_2023.select("pts", "trb", "ast", "stl", "blk").toPandas()

# Calcola e visualizza la matrice di correlazione
plt.figure(figsize=(10, 8))
correlation_matrix = stats_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice di Correlazione tra Statistiche di Base - Stagione 2023', fontsize=16)
plt.show()

In [ ]:
# Termina la sessione Spark
spark.stop()